# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [166]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [171]:
!ls

env_variable.json  sample_data


In [164]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [167]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [168]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

### TODO #1
Hoàn tất đoạn code sau

In [169]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [170]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [188]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](../img/google_service_account.png)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](../img/gservice_acc_json.png)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](../img/enable_api.png)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [172]:
!ls

env_variable.json  quickstart-313303-7fd9129f83f5.json	sample_data


In [175]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'quickstart-313303-7fd9129f83f5.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](../img/enable_api.png)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](../img/add_gsheet.png)

In [177]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](../img/slack_user_gsheet.png)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [187]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [183]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'created': 1600856703,
 'creator': 'U01BE2PR6LU',
 'id': 'C01B4PVGLVB',
 'is_archived': False,
 'is_channel': True,
 'is_ext_shared': False,
 'is_general': True,
 'is_group': False,
 'is_im': False,
 'is_member': False,
 'is_mpim': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'is_private': False,
 'is_shared': False,
 'name': 'general',
 'name_normalized': 'general',
 'num_members': 62,
 'parent_conversation': None,
 'pending_connected_team_ids': [],
 'pending_shared': [],
 'previous_names': [],
 'purpose': {'creator': 'U01BE2PR6LU',
  'last_set': 1600856703,
  'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.'},
 'shared_team_ids': ['T01B7SGGMLJ'],
 'topic': {'creator': '', 'last_set': 0, 'value': ''},
 'unlinked': 0}

### TODO#3 
* Tìm id của channel #atom-assignment2

### Slack API: List messages trong 1 channel

In [192]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [193]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [199]:
msg_ls[21]

{'attachments': [{'fallback': 'Google Colaboratory',
   'from_url': 'https://colab.research.google.com/drive/1VnMQdhpjKldwvJnjqpYO0sQkeaVbvNyB?usp=sharing',
   'id': 1,
   'image_bytes': 4534,
   'image_height': 250,
   'image_url': 'https://colab.research.google.com/img/colab_favicon_256px.png',
   'image_width': 250,
   'original_url': 'https://colab.research.google.com/drive/1VnMQdhpjKldwvJnjqpYO0sQkeaVbvNyB?usp=sharing',
   'service_icon': 'https://colab.research.google.com/img/favicon.ico?vrz=colab-20210422-060049-RC00_369841174',
   'service_name': 'colab.research.google.com',
   'title': 'Google Colaboratory',
   'title_link': 'https://colab.research.google.com/drive/1VnMQdhpjKldwvJnjqpYO0sQkeaVbvNyB?usp=sharing'}],
 'blocks': [{'block_id': '5uv',
   'elements': [{'elements': [{'text': 'Mình gửi bài assignment1:\nTask 1: ',
       'type': 'text'},
      {'type': 'link',
       'url': 'https://docs.google.com/spreadsheets/d/1XDHiv72ITLoY1aGYuc1LbqpVq20k72th4XRYPmqSAhI/edit?usp=sh

In [35]:
not_learners_id = ['U01BE2PR6LU']

In [72]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 04:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/danhpcv/
2021-05-10 04:35:39.003800 U01UMC08KL2 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/auslynnguyen/
2021-05-08 06:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 13:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/hoaintp/
2021-05-03 17:41:18.002400 U01U6JQB695 2 ['U01UMC0N5U2', 'U01U6JQB695'] 2021-05-11 03:10:06.001300 https://github.com/Tenderriver/
2021-05-03 08:51:19.001200 U01V00KHHHP 1 ['U01UMBZHU2W'] 2021-05-14 17:51:26.024600 https://github.com/saturn1101/
2021-04-26 06:53:56.000800 U01UMBZHU2W 2 ['U01UMJ451V1', 'U01UMBZHU2W'] 2021-05-14 17:52:35.025100 https://github.com/saturn1101/
2021-04-24 02:01:27.043200 U01VB63LRNC 1 ['U01UMJ451V1'] 2021-05-11 12:37:14.003100 https://github.com/AnhThuNguyenHuynh/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

-------------- HẾT PHẦN OPTION 2 ---------------------